In [7]:
import pymysql
import pandas as pd

### kakaotrend 전체기간동안의 경제keyword에 대한 검색량 counts정보 dataframe화

In [68]:
kakaotrends_df = pd.read_csv('./data/kakaotrend.csv', header=7, sep=',', skip_blank_lines = True)
kakaotrends_df = kakaotrends_df[0:145]
kakaotrends_df

,일,경제
0,2018-01-01,26
1,2018-01-08,24
2,2018-01-15,23
3,2018-01-22,21
4,2018-01-29,26
...,...,...
140,2020-09-07,29
141,2020-09-14,28
142,2020-09-21,28
143,2020-09-28,19


### DB연동

In [2]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [65]:
cur.execute("show databases")
print(cur.fetchall())
cur.execute("use CLI")

(('information_schema',), ('CLI',), ('MusicBank',), ('mysql',), ('mytestdb',), ('performance_schema',), ('sys',))


0

### DB table 생성

In [3]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS kakao_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL,
    cnt double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [8]:
cur.execute("desc kakao_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,cnt,double,YES,,None,


### kakaotrend의 경제keyword counting정보 주별로 저장

In [64]:
"""
"
날짜에서 year, month, day 추출
"""
def extract_year_and_month_day(period):
    year = period.split('-')[0]
    month = period.split('-')[1]
    day = period.split('-')[2]
    return year, month, day


In [70]:
"""
db table에 저장
"""
for i in range(len(df)):
    query = """ 
    Insert into kakao_counting (year, month, day, cnt) values (%d, %d, %d, %f) ;
    """
    year, month, day = extract_year_and_month_day(df.iloc[i].일)
    kakao_cnt = df.iloc[i].경제

    mystring = (query % (int(year), int(month), int(day), float(kakao_cnt)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 1, 1, 26.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 1, 8, 24.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 1, 15, 23.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 1, 22, 21.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 1, 29, 26.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 2, 5, 33.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 2, 12, 21.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 2, 19, 27.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 2, 26, 23.000000) ;
    
 
    Insert into kakao_counting (year, month, day, cnt) values (2018, 3, 5, 25.000000) ;
    
 
    Insert into kakao_counting (year, mont

In [71]:
# 확인
query = """ 
select * from kakao_counting ;
"""
cur.execute(query)
tmp=pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3,4
0,1,2018,1,1,26.0
1,2,2018,1,8,24.0
2,3,2018,1,15,23.0
3,4,2018,1,22,21.0
4,5,2018,1,29,26.0
...,...,...,...,...,...
140,141,2020,9,7,29.0
141,142,2020,9,14,28.0
142,143,2020,9,21,28.0
143,144,2020,9,28,19.0


In [72]:
conn.commit()
cur.close()
conn.close()